In [ ]:
#%pip3 install seaborn
#%pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

from PIL import Image
import torch
import torch.nn as nn
from torch import nn, save, load
from torch.optim import Adam
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
#import seaborn as sns

import pandas as pd
import numpy as np
import sqlite3

#from sklearn.metrics import mean_squared_error

from Sequences import Sequences
from ValueSequencer import HexagramValueSequencer

In [ ]:
def sigmoid(z):
  return 1 / (1 + np.exp(-z)) # aka 1 / 1 + e^-x

def test_sigmoid():
	# 👇 OUR "ARTIFICIAL" NEURON...
	input = 0.2 # fmr. called x
	weight = 1 # fmr. called m
	bias = 0 # # fmr. called b
	d_x = weight*input + bias
	a_x = sigmoid
	output = a_x(d_x)

	print(f"Input: {input}")
	print(f"Weight: {weight}")
	print(f"Bias: {bias}")
	print(f"Output: {output}")
	print()

In [ ]:

def GenerateSequence1(thvs, inputs):
	hvs1 = HexagramValueSequencer()
	hvs1._currentSequence = 0
	hvs1.First()
	p1 = 0
	for p in range(64):
		for s in range(64):
			hvs2 = hvs1.copy()

			for l in range(6):
				if ((hvs1.Value() < 32) and (((s & (1 << l)) >> l) == 0)) or ((hvs1.Value() >= 32) and (((s & (1 << l)) >> l) == 1)): # p -> hvs1.Value()
					hvs2.Trigram(l // 3).Line(l % 3).Next(False)

			#if (p * 64 + s < 64) or (p * 64 + s > 4031):
			#	print(hvs2.DescribeCast(True))

			#hvs2.Move()
			for l in range(6):
				v = hvs2.Trigram(l // 3).Line(l % 3).Value()
				v0 = 0
				v1 = 0
				v2 = 0
				if v == 1:
					v0 = 1
					v2 = 1
				if v == 2:
					v1 = 1
				if v == 3:
					v0 = 1
					v1 = 1
					v2 = 1
				thvs[p * 64 + s, l, 0] = v0
				thvs[p * 64 + s, l, 1] = v1 
				thvs[p * 64 + s, l, 2] = v2 

		c = 0
		for l in range(5, -1, -1):
			v = hvs1.Trigram(l // 3).Line(l % 3).Value()
			v0 = 0
			v1 = 0
			v2 = 0
			if v == 1:
				v0 = 1
				v2 = 1
			if v == 2:
				v1 = 1
			if v == 3:
				v0 = 1
				v1 = 1
				v2 = 1
			inputs[p, c] = v0# / 10
			inputs[p, c] = v1# / 10
			inputs[p, c] = v2# / 10
			c += 1 

		hvs1.Next()

	#print()	
	#print(thvs)

	#print(inputs)
	#print()	

	#print()	
	#hexagramArray.sort()

class YijingNN1(nn.Module):
	def __init__(self):
		super(YijingNN1, self).__init__()
		self.input_layer = nn.Linear(6, 64)
		
		self.layer_1 = nn.Linear(64, 1)

		#self.layer_1 = nn.Linear(64, 64)
		#self.layer_2 = nn.Linear(64, 1)
		
		# Sigmoid function for output
		self.sigmoid = nn.Sigmoid()

		# Initialize weights to 0
		#self.initialize_weights()

	def initialize_weights(self):
		nn.init.constant_(self.input_layer.weight, 0)
		nn.init.constant_(self.input_layer.bias, 0.5)
		nn.init.constant_(self.layer_1.weight, 0)
		nn.init.constant_(self.layer_1.bias, 0.5)

	def forward(self, x):
		x = torch.relu(self.input_layer(x))
		
		x = self.layer_1(x)
		
		#x = torch.relu(self.layer_1(x))
		#x = self.layer_2(x)
		
		x = self.sigmoid(x)
		return x

def train_yijing_model1():
	thvs = torch.zeros(4096, 6, 3, dtype=torch.int, requires_grad=False) # 2 ^ 6 = 64, 4 ^ 6 = 4096, 8 ^ 6 = 262,144
	inputs = torch.zeros(64, 6, dtype=torch.float32, requires_grad=False)

	labels = torch.tensor([
		[0.00],	[0.01],	[0.02],	[0.03],	[0.04],	[0.05],	[0.06],	[0.07],	[0.08],	[0.09],
		[0.10],	[0.11],	[0.12],	[0.13],	[0.14],	[0.15],	[0.16],	[0.17],	[0.18],	[0.19],
		[0.20],	[0.21],	[0.22],	[0.23],	[0.24],	[0.25],	[0.26],	[0.27],	[0.28],	[0.29],
		[0.30],	[0.31],	[0.32],	[0.33],	[0.34],	[0.35],	[0.36],	[0.37],	[0.38],	[0.39],
		[0.40],	[0.41],	[0.42],	[0.43],	[0.44],	[0.45],	[0.46],	[0.47],	[0.48],	[0.49],
		[0.50],	[0.51],	[0.52],	[0.53],	[0.54],	[0.55],	[0.56],	[0.57],	[0.58],	[0.59],
		[0.60],	[0.61],	[0.62],	[0.63],
	], dtype=torch.float32, requires_grad=False)
	
	GenerateSequence1(thvs, inputs)

	model = YijingNN1()

	# Define a loss function and an optimizer
	criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
	optimizer = optim.SGD(model.parameters(), lr=0.1)  # Stochastic Gradient Descent optimizer  0.001

	# Training loop
	num_epochs = 50000 #100000
	for epoch in range(num_epochs):
		# Zero the gradients
		optimizer.zero_grad()
		# Forward pass: compute the model output
		outputs = model(inputs)
		# Compute the loss
		loss = criterion(outputs, labels)
		# Backward pass: compute gradients
		loss.backward()
		# Update weights
		optimizer.step()
		# Print loss every 100 epochs
		if (epoch+1) % 10000 == 0:
			print(f'Epoch [{epoch+1}/{num_epochs}], Cost: {loss.item():.4f}')

	print()

	with open('yijing_model1_state.pt', 'wb') as f: 
		save(model.state_dict(), f) 

def test_yijing_model1():

	model = YijingNN1()

	with open('yijing_model1_state.pt', 'rb') as f: 
		model.load_state_dict(load(f, weights_only=False))  

	h1 = torch.tensor([[0, 0, 0, 0, 0, 0]], dtype=torch.float32)
	h2 = torch.tensor([[0, 0, 0, 0, 0, 1]], dtype=torch.float32)
	h3 = torch.tensor([[1, 0, 0, 0, 0, 0]], dtype=torch.float32)
	h4 = torch.tensor([[1, 1, 1, 1, 1, 0]], dtype=torch.float32)
	h5 = torch.tensor([[1, 1, 1, 1, 1, 1]], dtype=torch.float32)

	prediction1 = model(h1)
	prediction2 = model(h2)
	prediction3 = model(h3)
	prediction4 = model(h4)
	prediction5 = model(h5)
	print(f'Hexagram  0: {h1} Prediction: {prediction1.item() * 100:.0f}')
	print(f'Hexagram  1: {h2} Prediction: {prediction2.item() * 100:.0f}')
	print(f'Hexagram 32: {h3} Prediction: {prediction3.item() * 100:.0f}')
	print(f'Hexagram 62: {h4} Prediction: {prediction4.item() * 100:.0f}')
	print(f'Hexagram 63: {h5} Prediction: {prediction5.item() * 100:.0f}')
	print()

#train_yijing_model1()
test_yijing_model1()

In [ ]:
def GenerateSequence2(inputs, labels):
	hvs1 = HexagramValueSequencer()
	hvs1._currentSequence = 0
	hvs1.First()
	for p in range(64):
		for s in range(64):
			hvs2 = hvs1.copy()
			for l in range(6):
				#if ((hvs1.Value() < 32) and (((s & (1 << l)) >> l) == 0)) or ((hvs1.Value() >= 32) and (((s & (1 << l)) >> l) == 1)):
				if ((s & (1 << l)) >> l) == 1:
					hvs2.Trigram(l // 3).Line(l % 3).Next(False)

			c = 0
			for l in range(5, -1, -1):
				inputs[p * 64 + s, c] = hvs2.Trigram(l // 3).Line(l % 3).Value()
				c += 1 

			s1 = hvs2.DescribeCast(True).split(" ")
			#f = np.float64(float(s1[0])) / 100
			f = p / 100 + s / 10000
			labels[p * 64 + s] = f

			#if (p * 64 + s < 64) or (p * 64 + s > 4031):
			#	print(f'{hvs2.DescribeCast(True)}, {f:.4f}')

		hvs1.Next()

	#print()	
	#print(thvs)

	#print(inputs)
	#print()	

	#print()	
	#hexagramArray.sort()

class YijingNN2(nn.Module):
	def __init__(self):
		super(YijingNN2, self).__init__()
		self.input_layer = nn.Linear(6, 4096, dtype=torch.float32)
		
		self.layer_1 = nn.Linear(4096, 1, dtype=torch.float32)

		#self.layer_1 = nn.Linear(64, 64)
		#self.layer_2 = nn.Linear(64, 1)
		
		# Sigmoid function for output
		self.sigmoid = nn.Sigmoid()

		# Initialize weights to 0
		#self._initialize_weights()

	def x_initialize_weights(self):
		nn.init.constant_(self.input_layer.weight, 0)
		nn.init.constant_(self.input_layer.bias, 0)
		nn.init.constant_(self.layer_1.weight, 0)
		nn.init.constant_(self.layer_1.bias, 0)

	def forward(self, x):
		x = torch.relu(self.input_layer(x))
		
		x = self.layer_1(x)
		
		#x = torch.relu(self.layer_1(x))
		#x = self.layer_2(x)
		
		x = self.sigmoid(x)
		return x

def train_yijing_model2():
	inputs = torch.zeros(4096, 6, dtype=torch.float32, requires_grad=False).to('cpu')
	labels = torch.zeros(4096, 1, dtype=torch.float32, requires_grad=False).to('cpu')

	GenerateSequence2(inputs, labels)
	print(inputs)
	print(labels)

	model = YijingNN2().to('cpu')

	# Define a loss function and an optimizer
	criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
	optimizer = optim.SGD(model.parameters(), lr=0.1)  # Stochastic Gradient Descent optimizer  0.001

	# Training loop
	num_epochs = 10_000 #50_000
	for epoch in range(num_epochs):
		# Zero the gradients
		optimizer.zero_grad()
		# Forward pass: compute the model output
		outputs = model(inputs)
		# Compute the loss
		loss = criterion(outputs, labels)
		# Backward pass: compute gradients
		loss.backward()
		# Update weights
		optimizer.step()
		# Print loss every 100 epochs
		if (epoch+1) % 1_000 == 0:
			print(f'Epoch [{epoch+1}/{num_epochs}], Cost: {loss.item():.4f}')

	print()

	with open('yijing_model2_state.pt', 'wb') as f: 
		save(model.state_dict(), f) 

def test_yijing_model2():

	model = YijingNN2()

	with open('yijing_model2_state.pt', 'rb') as f: 
		model.load_state_dict(load(f, weights_only=False))  

	h1 = torch.tensor([[2, 2, 2, 2, 2, 2]], dtype=torch.float32)
	h2 = torch.tensor([[2, 2, 2, 0, 0, 3]], dtype=torch.float32)
	h3 = torch.tensor([[3, 3, 3, 2, 2, 2]], dtype=torch.float32)
	h4 = torch.tensor([[1, 1, 1, 3, 3, 0]], dtype=torch.float32)
	h5 = torch.tensor([[1, 1, 1, 1, 1, 1]], dtype=torch.float32)

	prediction1 = model(h1)
	prediction2 = model(h2)
	prediction3 = model(h3)
	prediction4 = model(h4)
	prediction5 = model(h5)
	print(f'Hexagram 0: {h1} Prediction: {prediction1.item():.4f}')
	print(f'Hexagram 1.123: {h2} Prediction: {prediction2.item():.4f}')
	print(f'Hexagram 57.456: {h3} Prediction: {prediction3.item():.4f}')
	print(f'Hexagram 62.123: {h4} Prediction: {prediction4.item():.4f}')
	print(f'Hexagram 63: {h5} Prediction: {prediction5.item():.4f}')
	print()

#train_yijing_model2()
test_yijing_model2()

In [ ]:
"""
class EegNN(nn.Module):
	def __init__(self):
		super(EegNN, self).__init__()
		self.input_layer = nn.Linear(20, 100)
		#self.layer_1 = nn.Linear(100, 1)
		self.layer_1 = nn.Linear(100, 20)
		self.layer_2 = nn.Linear(20, 10)
		self.layer_3 = nn.Linear(10, 1)
		# Sigmoid function for output
		self.sigmoid = nn.Sigmoid()

	def forward(self, x):
		x = torch.relu(self.input_layer(x))
		#x = self.layer_1(x)
		x = torch.relu(self.layer_1(x))
		x = torch.relu(self.layer_2(x))
		x = self.layer_3(x)
		x = self.sigmoid(x)
		return x

		
class EegNN(nn.Module):
	def __init__(self):
		super(EegNN, self).__init__()
		self.model = nn.Sequential(
			nn.Linear(20, 100),
			nn.ReLU(),
			nn.Linear(100, 20),
			nn.ReLU(),
			nn.Linear(20, 10),
			nn.ReLU(),
			nn.Linear(10, 1),
			nn.Sigmoid()
			)

	def forward(self, x):
		return self.model(x)

class EegNN(nn.Module):
	def __init__(self):
		super(EegNN, self).__init__()
		self.input_layer = nn.Linear(20, 100, dtype=torch.float32)
		
		#self.layer_1 = nn.Linear(100, 1, dtype=torch.float32)

		self.layer_1 = nn.Linear(100, 10)
		self.layer_2 = nn.Linear(10, 1)
		
		# Sigmoid function for output
		self.sigmoid = nn.Sigmoid()

		# Initialize weights to 0
		#self._initialize_weights()

	def x_initialize_weights(self):
		nn.init.constant_(self.input_layer.weight, 0)
		nn.init.constant_(self.input_layer.bias, 0)
		nn.init.constant_(self.layer_1.weight, 0)
		nn.init.constant_(self.layer_1.bias, 0)

	def forward(self, x):
		x = torch.relu(self.input_layer(x))
		
		#x = self.layer_1(x)
		
		x = torch.relu(self.layer_1(x))
		x = self.layer_2(x)
		
		x = self.sigmoid(x)
		return x
"""

class EegNN(nn.Module):
	def __init__(self):
		super(EegNN, self).__init__()
		self.model = nn.Sequential(
			nn.Linear(20, 100),
			nn.ReLU(),
			nn.Linear(100, 1),
			nn.Sigmoid()
			)

	def forward(self, x):
		return self.model(x)

def GenerateData(inputs, labels):

	Delta_TP9_i = 0
	Delta_AF7_i = 1
	Delta_AF8_i = 2
	Delta_TP10_i = 3
	Theta_TP9_i = 4
	Theta_AF7_i = 5
	Theta_AF8_i = 6
	Theta_TP10_i = 7
	Alpha_TP9_i = 8
	Alpha_AF7_i = 9
	Alpha_AF8_i = 10
	Alpha_TP10_i = 11
	Beta_TP9_i = 12
	Beta_AF7_i = 13
	Beta_AF8_i = 14
	Beta_TP10_i = 15
	Gamma_TP9_i = 16
	Gamma_AF7_i = 17
	Gamma_AF8_i = 18
	Gamma_TP10_i = 19

	Delta_AF7_min = -0.5
	Delta_AF8_min = -0.5
	Delta_AF7_v = 0.0
	Delta_AF8_v = 0.0

	Delta_TP9_min = -0.1
	Delta_TP10_min = -0.1
	Delta_TP9_v = 0.2
	Delta_TP10_v = 0.2
	
	Theta_AF7_min = -0.5
	Theta_AF8_min = -0.5
	Theta_AF7_v = 0.0
	Theta_AF8_v = 0.0
	
	Theta_TP9_min = -0.1
	Theta_TP10_min = -0.1
	Theta_TP9_v = 0.2
	Theta_TP10_v = 0.2
	
	Alpha_AF7_min = 0.0
	Alpha_AF8_min = 0.0
	Alpha_AF7_v = 0.5
	Alpha_AF8_v = 0.5
	
	Alpha_TP9_min = 0.4
	Alpha_TP10_min = 0.4
	Alpha_TP9_v = 0.8
	Alpha_TP10_v = 0.8
	
	Beta_AF7_min = 0.0
	Beta_AF8_min = 0.0
	Beta_AF7_v = 1.4
	Beta_AF8_v = 1.4
	
	Beta_TP9_min = 0.0
	Beta_TP10_min = 0.0
	Beta_TP9_v = 0.5
	Beta_TP10_v = 0.5
	
	Gamma_AF7_min = 0.0
	Gamma_AF8_min = 0.0
	Gamma_AF7_v = 1.8
	Gamma_AF8_v = 1.8

	Gamma_TP9_min = 0.0
	Gamma_TP10_min = 0.0
	Gamma_TP9_v = 0.4
	Gamma_TP10_v = 0.4
	
	for p in range(100):
		inputs[p, Delta_TP9_i] = Delta_TP9_v if p % 2 == 0 else Delta_TP9_min
		inputs[p, Delta_AF7_i] = Delta_AF7_v if p % 2 == 0 else Delta_AF7_min
		inputs[p, Delta_AF8_i] = Delta_AF8_v if p % 2 == 0 else Delta_AF8_min
		inputs[p, Delta_TP10_i] = Delta_TP10_v if p % 2 == 0 else Delta_TP10_min
		inputs[p, Theta_TP9_i] = Theta_TP9_v if p % 2 == 0 else Theta_TP9_min
		inputs[p, Theta_AF7_i] = Theta_AF7_v if p % 2 == 0 else Theta_AF7_min
		inputs[p, Theta_AF8_i] = Theta_AF8_v if p % 2 == 0 else Theta_AF8_min
		inputs[p, Theta_TP10_i] = Theta_TP10_v if p % 2 == 0 else Theta_TP10_min
		inputs[p, Alpha_TP9_i] = Alpha_TP9_v if p % 2 == 0 else Alpha_TP9_min
		inputs[p, Alpha_AF7_i] = Alpha_AF7_v if p % 2 == 0 else Alpha_AF7_min
		inputs[p, Alpha_AF8_i] = Alpha_AF8_v if p % 2 == 0 else Alpha_AF8_min
		inputs[p, Alpha_TP10_i] = Alpha_TP10_v if p % 2 == 0 else Alpha_TP10_min

		inputs[p, Beta_TP9_i] = Beta_TP9_v * p / 100 if Beta_TP9_v * p / 100 > Beta_TP9_min else Beta_TP9_min
		inputs[p, Beta_AF7_i] = Beta_AF7_v * p / 100 if Beta_AF7_v * p / 100 > Beta_AF7_min else Beta_AF7_min
		inputs[p, Beta_AF8_i] = Beta_AF8_v * p / 100 if Beta_AF8_v * p / 100 > Beta_AF8_min else Beta_AF8_min
		inputs[p, Beta_TP10_i] = Beta_TP10_v * p / 100 if Beta_AF8_v * p / 100 > Beta_AF8_min else Beta_AF8_min
		inputs[p, Gamma_TP9_i] = Gamma_TP9_v * p / 100 if Gamma_TP9_v * p / 100 > Gamma_TP9_min else Gamma_TP9_min
		inputs[p, Gamma_AF7_i] = Gamma_AF7_v * p / 100 if Gamma_AF7_v * p / 100 > Gamma_AF7_min else Gamma_AF7_min
		inputs[p, Gamma_AF8_i] = Gamma_AF8_v * p / 100 if Gamma_AF8_v * p / 100 > Gamma_AF8_min else Gamma_AF8_min
		inputs[p, Gamma_TP10_i] = Gamma_TP10_v * p / 100 if Gamma_TP10_v * p / 100 > Gamma_TP10_min else Gamma_TP10_min

		labels[p] = p / 100

	#print()	

#['TimeStamp','Gyro_X','Gyro_Y','Gyro_Z','Accelerometer_X','Accelerometer_Y','Accelerometer_Z','AUX_RIGHT','HSI_TP9','HSI_AF7',
#'HSI_AF8','HSI_TP10','RAW_TP9','RAW_AF7','RAW_AF8','RAW_TP10','HeadBandOn','Battery','Elements','Mellow','Concentration']

filename = 'C:\\Users\\Stephen Vivash\\Documents\\Yijing\\Muse\\2024-07-29-17-24-00-Muse.csv'
#filename = 'C:\\Users\\Stephen Vivash\\Documents\\Yijing\\Muse\\2024-08-10-18-07-51-Muse.csv'
all_columns = ['Delta_TP9','Delta_AF7','Delta_AF8','Delta_TP10','Theta_TP9','Theta_AF7','Theta_AF8','Theta_TP10','Alpha_TP9','Alpha_AF7','Alpha_AF8','Alpha_TP10',
		'Beta_TP9','Beta_AF7','Beta_AF8','Beta_TP10','Gamma_TP9','Gamma_AF7','Gamma_AF8','Gamma_TP10']
graph_columns = [   'Delta_TP9'   ,'Alpha_AF7','Alpha_AF8','Beta_AF7','Beta_AF8','Gamma_AF7','Gamma_AF8']
#graph_columns = ['Alpha_TP9','Alpha_AF7','Alpha_AF8','Alpha_TP10','Beta_TP9','Beta_AF7','Beta_AF8','Beta_TP10','Gamma_TP9','Gamma_AF7','Gamma_AF8','Gamma_TP10']) 
tail = 11000

def train_eeg_model1():

	inputs = torch.zeros(100, 20, dtype=torch.float32, requires_grad=False).to('cpu')
	labels = torch.zeros(100, 1, dtype=torch.float32, requires_grad=False).to('cpu')
	GenerateData(inputs, labels)
	#df = pd.DataFrame(inputs.numpy(), columns=all_columns)
	#df.plot(kind='line',legend=True,figsize=(20, 10))
	#print(inputs)
	#print(labels)

	data = pd.read_csv(filename, usecols=all_columns) 
	data = data.tail(tail)
	data = data.head(100)
	data.plot(kind='line',legend=True,figsize=(20, 10))

	# Convert NumPy array to PyTorch tensor
	#inputs = torch.from_numpy(data.to_numpy()).float()
	#print(inputs)

	# Normalize the input data
	mean = inputs.mean(dim=0, keepdim=True)
	#print('mean', mean)
	std = inputs.std(dim=0, keepdim=True)
	#print('std', std)
	#inputs = (inputs - mean) / std
	#print('inputs', inputs)
	print()

	model = EegNN().to('cpu')

	# Define a loss function and an optimizer
	criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
	optimizer = optim.SGD(model.parameters(), lr=0.1)  # Stochastic Gradient Descent optimizer  0.001

	# Training loop
	num_epochs = 100_000 #50_000 # 1_000_000
	for epoch in range(num_epochs):
		# Zero the gradients
		optimizer.zero_grad()
		# Forward pass: compute the model output
		outputs = model(inputs)
		# Compute the loss
		loss = criterion(outputs, labels)
		# Backward pass: compute gradients
		loss.backward()
		# Update weights
		optimizer.step()
		# Print loss every 100 epochs
		if (epoch+1) % 10_000 == 0: # 100_000 == 0:
			print(f'Epoch [{epoch+1}/{num_epochs}], Cost: {loss.item():.4f}')

	print()

	with open('eeg_model1_state.pt', 'wb') as f: 
		save(model.state_dict(), f) 

def test_eeg_model1():
	df1 = pd.read_csv(filename,	usecols=all_columns) 
	df1 = df1.tail(tail)
	df1 = df1.head(100)

	inputs = torch.zeros(100, 20, dtype=torch.float32, requires_grad=False).to('cpu')
	labels = torch.zeros(100, 1, dtype=torch.float32, requires_grad=False).to('cpu')
	GenerateData(inputs, labels)
	
	model = EegNN().to('cpu')

	# Convert NumPy array to PyTorch tensor
	inputs = torch.from_numpy(df1.to_numpy()).float().to('cpu')
	#print(inputs)

	# Normalize the input data
	#mean = inputs.mean(dim=0, keepdim=True)
	#print('mean', mean)
	#std = inputs.std(dim=0, keepdim=True)
	#print('std', std)
	#inputs = (inputs - mean) / std
	#print('inputs', inputs)
	#print()

	with open('eeg_model1_state.pt', 'rb') as f: 
		model.load_state_dict(load(f, weights_only=False))  

	#df2 = pd.DataFrame(inputs.numpy(), columns=all_columns)
	df2 = pd.read_csv(filename,usecols=graph_columns) 
	df2 = df2.tail(tail)
	df2 = df2.head(100)
	n2 = df2.to_numpy()
	df2.columns.values[0] = 'Prediction'

	for i in range(100):
		e1 = inputs[i]
		#e1 = (e1 - mean) / std
		p1 = model(e1)
		n2[i][0] = p1.item()
		#print(f'Prediction: {p1.item():.2f}')

	#df2.to_csv('eeg_model1.csv', index=False)

	df2.plot(kind='line',legend=True,figsize=(20, 10), linewidth=3.0)

	print()

#train_eeg_model1()
test_eeg_model1()